In [14]:
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [61]:
datapath_2016 = "./zillow-data/merged_new2016.csv"
datapath_2017 = "./zillow-data/merged_new2017.csv"

test_datapath_2016 = "./zillow-data/properties_2016_new v2.csv"
test_datapath_2017 = "./zillow-data/properties_2017_new.csv"

In [75]:
# modify 'transactiondate' of df to keep only the month
def parse_transactiondate(df):
    # modify transactiondate col to keep only the month
    new_transactiondate = []
    for data in df['transactiondate']:
        temp = data.split("-")
        new_transactiondate.append(int(temp[1])) # keep only month

    new_transactiondate = np.array(new_transactiondate)
    new_transactiondate = new_transactiondate.reshape(new_transactiondate.shape[0], 1)

    df['transactiondate'] = new_transactiondate

# return label index, feature index list
# assumes that df label_is_first
def split_on_label(df):
    return df.columns[0], df.columns[1:]

def gen_testdata(df, new_transactiondate):
    df_copy = df.copy()
    # df_copy['transactiondate'] = new_transactiondate
    df_copy.insert(0, 'transactiondate', new_transactiondate)
    return df_copy

In [67]:
train_data2016 = pd.read_csv(datapath_2016, index_col=0)
train_data2017 = pd.read_csv(datapath_2017, index_col=0)

# --- TEMP Fixes for train_data2016 (not well formatted) ---
# fix Unamed: 0 at col index 3
fix_col = train_data2016.columns.str.contains('unnamed',case = False)
train_data2016 = train_data2016.drop(train_data2016.columns[fix_col], axis=1)
# --- END ---

parcelid16 = train_data2016['parcelid']
parcelid17 = train_data2017['parcelid']

# drop parcelid (index of properties features)
train_data2016 = train_data2016.drop('parcelid', axis=1)
train_data2017 = train_data2017.drop('parcelid', axis=1)

label16, feature16 = split_on_label(train_data2016)
label17, feature17 = split_on_label(train_data2017)

print("[2016] num of features:", len(feature16))
print("[2016] num of instances:", train_data2016.shape[0])

print("[2017] num of features:", len(feature17))
print("[2017] num of instances:", train_data2017.shape[0])

# modify 'transactiondate' to keep only the month
parse_transactiondate(train_data2016)
parse_transactiondate(train_data2017)

# training data and label for model 2016
X_train16 = train_data2016[feature16]
y_train16 = train_data2016[label16]

# training data and label for model 2017
X_train17 = train_data2017[feature17]
y_train17 = train_data2017[label17]


[2016] num of features: 31
[2016] num of instances: 90275
[2017] num of features: 31
[2017] num of instances: 77613


In [41]:
# Ridge Regression

def handle_regrRidge(X_train, y_train):
    regrRidge = linear_model.RidgeCV(alphas=(0.1, 1.0, 10.0), normalize=True, cv=10, 
                                 scoring='neg_mean_squared_error')
    regrRidge.fit(X_train, y_train)
    y_pred = regrRidge.predict(X_train)

    print("RidgeCV alpha(lambda):{:.2f}".format(regrRidge.alpha_))
    print("Training MSE: {:.6f}".format(mean_squared_error(y_train, y_pred)))
    print("Training R^2: {:.6f}".format(r2_score(y_train, y_pred)))
    
    return regrRidge
    

print("Ridge Regression for 2016 train data")
regrRidge16 = handle_regrRidge(X_train16, y_train16)
print("\n")

print("Ridge Regression for 2017 train data")
regrRidge17 = handle_regrRidge(X_train17, y_train17)
print("\n")

Ridge Regression for 2016 train data
RidgeCV alpha(lambda):0.10
Training MSE: 0.025865
Training R^2: 0.003140


Ridge Regression for 2017 train data
RidgeCV alpha(lambda):1.00
Training MSE: 0.029127
Training R^2: 0.002564




In [33]:
# LASSO

def handle_Lasso(X_train, y_train):
    regrLasso = linear_model.LassoCV(eps=0.001, n_alphas=100, normalize=True, cv=10, max_iter=50000)
    regrLasso.fit(X_train, y_train)
    y_pred = regrLasso.predict(X_train)
    
    print("LassoCV alpha(lambda): {:.6f}".format(regrLasso.alpha_))
    print("Mean squared error: {:.6f}".format(mean_squared_error(y_train, y_pred)))
    print("R^2: {:.6f}".format(r2_score(y_train, y_pred)))
    
    return regrLasso

# return feature given the coef values from model
def get_reduced_features(coef, features):
    sel_features = []
    for i in range(len(coef)):
        if coef[i] != 0:
            sel_features.append(features[i])
    return sel_features

print("Lasso Regression for 2016 train data")
regrLasso16 = handle_Lasso(X_train16, y_train16)
print("\n")

print("Lasso Regression for 2017 train data")
regrLasso17 = handle_Lasso(X_train17, y_train17)
print("\n")

# selected features from Lasso
sel_features16 = get_reduced_features(regrLasso16.coef_, feature)
sel_features17 = get_reduced_features(regrLasso17.coef_, feature)

print("[2016] Selected Features[{}]:".format(len(sel_features16)))
for i in sel_features16:
    print(i)
print("\n")

print("[2017] Selected Features[{}]:".format(len(sel_features17)))
for i in sel_features17:
    print(i)
print("\n")

Lasso Regression for 2016 train data
LassoCV alpha(lambda): 0.000002
Mean squared error: 0.025873
R^2: 0.002827


Lasso Regression for 2017 train data
LassoCV alpha(lambda): 0.000002
Mean squared error: 0.029103
R^2: 0.003399


[2016] Selected Features[13]:
transactiondate
calculatedfinishedsquarefeet
heatingorsystemtypeid
propertylandusetypeid
regionidzip
yearbuilt
landtaxvaluedollarcnt
censustractandblock
haspoolornot
regionidneighborhood
unitcnt
lotsizesquarefeet
finishedfloor1squarefeet


[2017] Selected Features[19]:
transactiondate
bedroomcnt
calculatedfinishedsquarefeet
fireplacecnt
garagetotalsqft
latitude
longitude
propertylandusetypeid
rawcensustractandblock
regionidcity
regionidcounty
regionidzip
assessmentyear
landtaxvaluedollarcnt
taxamount
haspoolornot
unitcnt
roomcnt
finishedfloor1squarefeet




In [35]:
# XGBoost fit Regression Decision Tree

def handle_XGBoost(X_train, y_train):
    # alpha values from 0.01 to 1000 up to 100 values log scale
    alphalist = np.logspace(-2, 4, 10, endpoint=True)

    mse_dict = {}
    kf = KFold(n_splits=5, shuffle=True)
    # cross-validate on alpha (regularization strenght) from alphalist
    for i in alphalist:
        bst = xgb.sklearn.XGBRegressor(reg_alpha=i)
        bst.fit(X_train, y_train)
        score = cross_val_score(bst, X_train, y_train, cv=kf, scoring='neg_mean_squared_error').mean()
        mse_dict[i] = -score

    opt_a = min(mse_dict, key= mse_dict.get)

    print("alpha (lambda) from CV: {:.6}".format(opt_a))

    # obtain train MSE with CV'ed alpha value
    bst = xgb.sklearn.XGBRegressor(reg_alpha=opt_a)
    bst.fit(X_train, y_train)
    y_pred = bst.predict(X_train)
    print("Train MSE: {:.6}".format(mean_squared_error(y_train, y_pred)))
    
    return bst

print("XGBoost Model 2016:")
bst16 = handle_XGBoost(X_train16, y_train16)
print("\n")

print("XGBoost Model 2017:")
bst17 = handle_XGBoost(X_train17, y_train17)
print("\n")

XGBoost Model 2016:
alpha (lambda) from CV: 4.64159
Train MSE: 0.0250956


XGBoost Model 2017:
alpha (lambda) from CV: 1.0
Train MSE: 0.0278357




In [76]:
# read in test
test_data2016 = pd.read_csv(test_datapath_2016, index_col=0)
test_data2017 = pd.read_csv(test_datapath_2017, index_col=0)

print("[2016] num of instances: ", test_data2016.shape[0])
print("[2017] num of instances: ", test_data2017.shape[0])

# save parcelid for merge 
test_parcelid16 = test_data2016['parcelid']
test_parcelid17 = test_data2017['parcelid']

# drop parcelid col
test_data2016 = test_data2016.drop('parcelid', axis=1)
test_data2017 = test_data2017.drop('parcelid', axis=1)

# generate transaction date
test10_16 = np.repeat(10, test_data2016.shape[0])
test11_16 = np.repeat(11, test_data2016.shape[0])
test12_16 = np.repeat(12, test_data2016.shape[0])

test10_17 = np.repeat(10, test_data2017.shape[0])
test11_17 = np.repeat(11, test_data2017.shape[0])
test12_17 = np.repeat(12, test_data2017.shape[0])

# get new test data with transaction date
X_test10_16 = gen_testdata(test_data2016, test10_16)
X_test11_16 = gen_testdata(test_data2016, test11_16)
X_test12_16 = gen_testdata(test_data2016, test12_16)

X_test10_17 = gen_testdata(test_data2017, test10_17)
X_test11_17 = gen_testdata(test_data2017, test11_17)
X_test12_17 = gen_testdata(test_data2017, test12_17)


/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


[2016] num of instances:  2985217
[2017] num of instances:  2985217


In [83]:
# predict on test data

y_pred10_16 = bst16.predict(X_test10_16)
y_pred11_16 = bst16.predict(X_test11_16)
y_pred12_16 = bst16.predict(X_test12_16)

y_pred10_17 = bst17.predict(X_test10_17)
y_pred11_17 = bst17.predict(X_test11_17)
y_pred12_17 = bst17.predict(X_test12_17)


In [87]:
test_dict_16 = {'Parcelid': test_parcelid16, '201610': y_pred10_16, '201611': y_pred11_16, '201612': y_pred12_16}
test_dict_17 = {'Parcelid': test_parcelid17, '201710': y_pred10_17, '201711': y_pred11_17, '201712': y_pred12_17}

df_test_16 = pd.DataFrame(data=test_dict_16)
df_test_17 = pd.DataFrame(data=test_dict_17)

df_merged = df_test_16.merge(df_test_17, left_on='Parcelid', right_on='Parcelid', how='outer')
print(df_merged)

          Parcelid    201610    201611    201612    201710    201711    201712
0         10754147  0.084751  0.084751  0.084751  0.104134  0.104134  0.104134
1         10759547  0.030246  0.030246  0.030246  0.096874  0.096874  0.096874
2         10843547  0.052090  0.052090  0.052090  0.106128  0.106128  0.106128
3         10859147  0.020549  0.020549  0.020549  0.088284  0.088284  0.088284
4         10879947  0.001912  0.001912  0.001912  0.009659  0.009659  0.009659
5         10898347  0.014306  0.014306  0.014306  0.022779  0.022779  0.022779
6         10933547  0.000933  0.000933  0.000933  0.006954  0.006954  0.006954
7         10940747  0.017823  0.017823  0.017823  0.028543  0.028543  0.028543
8         10954547  0.084149  0.084149  0.084149  0.064016  0.064016  0.064016
9         10976347  0.008127  0.008127  0.008127  0.031002  0.031002  0.031002
10        11073947  0.008052  0.008052  0.008052  0.019067  0.019067  0.019067
11        11114347  0.088350  0.088350  0.088350  0.

In [90]:
# handle submission file
submitfile = "./zillow-data/sample_submission.csv"

submit_df = pd.read_csv(submitfile)
print(submit_df.shape[0], submit_df.shape[1])

2985217 7


In [92]:
df_merged.to_csv("./zillow-data/draft_submission1.csv", index=False)